# 🚀 AI Fashion Assistant v2.0 - Project Initialization

**Phase 0: Setup**

This notebook initializes the project structure and validates the environment.

---

## 📋 Tasks

1. ✅ Mount Google Drive
2. ✅ Create folder structure
3. ✅ Copy data from v1
4. ✅ Validate environment
5. ✅ Install dependencies
6. ✅ Initialize Git
7. ✅ Test imports

---

In [1]:
# ============================================================
# 1) MOUNT GOOGLE DRIVE
# ============================================================

from google.colab import drive
import os

print("🔗 Mounting Google Drive...")
drive.mount("/content/drive", force_remount=False)
print("✅ Drive mounted!")

# Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

🔗 Mounting Google Drive...
Mounted at /content/drive
✅ Drive mounted!
NVIDIA A100-SXM4-40GB, 40960 MiB


In [2]:
# ============================================================
# 2) SET PATHS
# ============================================================

from pathlib import Path

# New project path
PROJECT_ROOT = Path("/content/drive/MyDrive/ai_fashion_assistant_v2")

# Old project path (for data copying)
OLD_PROJECT = Path("/content/drive/MyDrive/ai_fashion_assistant_v1")

print(f"📁 New Project: {PROJECT_ROOT}")
print(f"📁 Old Project: {OLD_PROJECT}")

# Verify new folder exists
if not PROJECT_ROOT.exists():
    print("❌ Project folder does not exist!")
    print(f"   Please create: {PROJECT_ROOT}")
    raise FileNotFoundError(f"Create {PROJECT_ROOT} in Google Drive")
else:
    print("✅ Project folder exists!")

📁 New Project: /content/drive/MyDrive/ai_fashion_assistant_v2
📁 Old Project: /content/drive/MyDrive/ai_fashion_assistant_v1
✅ Project folder exists!


In [3]:
# ============================================================
# 3) CREATE FOLDER STRUCTURE
# ============================================================

print("📂 Creating folder structure...\n")

folders = [
    # Data
    "data/raw",
    "data/processed",
    "data/ground_truth",
    "data/schemas",
    "data/user_profiles",

    # Embeddings
    "embeddings/text",
    "embeddings/image",
    "embeddings/hybrid",
    "embeddings/user",
    "embeddings/configs",

    # Models
    "models/fusion",
    "models/reranker",
    "models/personalization",
    "models/checkpoints",

    # Indexes
    "indexes",

    # Source code
    "src",
    "api",
    "api/endpoints",
    "ui",
    "ui/components",
    "ui/assets",

    # Configs
    "configs",

    # Scripts
    "scripts",

    # Tests
    "tests",

    # Docs
    "docs",
    "docs/results",

    # Experiments
    "experiments",

    # Logs
    "logs"
]

for folder in folders:
    folder_path = PROJECT_ROOT / folder
    folder_path.mkdir(parents=True, exist_ok=True)
    print(f"  ✅ {folder}")

print("\n✅ Folder structure created!")

📂 Creating folder structure...

  ✅ data/raw
  ✅ data/processed
  ✅ data/ground_truth
  ✅ data/schemas
  ✅ data/user_profiles
  ✅ embeddings/text
  ✅ embeddings/image
  ✅ embeddings/hybrid
  ✅ embeddings/user
  ✅ embeddings/configs
  ✅ models/fusion
  ✅ models/reranker
  ✅ models/personalization
  ✅ models/checkpoints
  ✅ indexes
  ✅ src
  ✅ api
  ✅ api/endpoints
  ✅ ui
  ✅ ui/components
  ✅ ui/assets
  ✅ configs
  ✅ scripts
  ✅ tests
  ✅ docs
  ✅ docs/results
  ✅ experiments
  ✅ logs

✅ Folder structure created!


In [5]:
# ============================================================
# 4) COPY DATA FROM V1 (IMPROVED - WITH SYMLINK)
# ============================================================

import shutil
import os

print("📋 Checking for data in old project...\n")

if OLD_PROJECT.exists():
    # Check for data/raw
    old_raw = OLD_PROJECT / "data/raw"
    new_raw = PROJECT_ROOT / "data/raw"

    if old_raw.exists():
        print("  Found data/raw in old project")

        # Copy styles.csv (small file, no problem)
        old_styles = old_raw / "styles.csv"
        if old_styles.exists():
            print("  Copying styles.csv...")
            shutil.copy2(old_styles, new_raw / "styles.csv")
            print("    ✅ styles.csv copied")

        # 🔥 IMAGES: USE SYMLINK INSTEAD OF COPY
        # This is much faster and doesn't duplicate data
        old_images = old_raw / "images"
        new_images = new_raw / "images"

        if old_images.exists():
            if new_images.exists():
                print("    ⚠️ Images already exist, skipping")
            else:
                # Option A: Create symlink (RECOMMENDED - instant!)
                try:
                    print("  Creating symlink to images folder (instant!)...")
                    os.symlink(str(old_images), str(new_images))
                    print("    ✅ Images symlink created")
                except Exception as e:
                    print(f"    ⚠️ Symlink failed: {e}")
                    print("    💡 Alternative: We'll reference old path directly")
                    # Option B: Just use old path in code
                    IMAGE_DIR = old_images

        # Verify
        if (new_raw / "images").exists() or old_images.exists():
            test_images = list(old_images.glob("*.jpg"))[:5]
            print(f"    ✅ Images accessible: {len(test_images)} samples verified")

    print("\n✅ Data setup completed!")

else:
    print("⚠️ Old project not found")
    print("   You'll need to manually add data to data/raw/")
    print("   Required files:")
    print("     - data/raw/styles.csv")
    print("     - data/raw/images/")

# ============================================================
# IMPORTANT: Update image path in config
# ============================================================

# If symlink worked, use new path
if (PROJECT_ROOT / "data/raw/images").exists():
    IMAGE_DIR = PROJECT_ROOT / "data/raw/images"
    print(f"\n✅ Image directory: {IMAGE_DIR}")
else:
    # Otherwise, use old path directly
    IMAGE_DIR = OLD_PROJECT / "data/raw/images"
    print(f"\n💡 Using old image directory: {IMAGE_DIR}")

print(f"   Total images: {len(list(IMAGE_DIR.glob('*.jpg')))}")

📋 Checking for data in old project...

  Found data/raw in old project
  Creating symlink to images folder (instant!)...
    ✅ Images symlink created
    ✅ Images accessible: 5 samples verified

✅ Data setup completed!

✅ Image directory: /content/drive/MyDrive/ai_fashion_assistant_v2/data/raw/images
   Total images: 44192


In [6]:
# ============================================================
# 5) VALIDATE DATA
# ============================================================

import pandas as pd

print("🔍 Validating data...\n")

styles_path = PROJECT_ROOT / "data/raw/styles.csv"
images_dir = PROJECT_ROOT / "data/raw/images"

# Check styles.csv
if styles_path.exists():
    df = pd.read_csv(styles_path, nrows=5)
    print(f"✅ styles.csv found: {len(pd.read_csv(styles_path))} rows")
    print(f"   Columns: {list(df.columns)}")
else:
    print("❌ styles.csv not found!")
    print(f"   Expected at: {styles_path}")

# Check images
if images_dir.exists():
    image_files = list(images_dir.glob("*.jpg"))
    print(f"✅ Images folder found: {len(image_files)} images")
else:
    print("❌ Images folder not found!")
    print(f"   Expected at: {images_dir}")

print("\n✅ Data validation completed!")

🔍 Validating data...

❌ styles.csv not found!
   Expected at: /content/drive/MyDrive/ai_fashion_assistant_v2/data/raw/styles.csv
✅ Images folder found: 44192 images

✅ Data validation completed!


In [7]:
# ============================================================
# 6) INSTALL DEPENDENCIES
# ============================================================

print("📦 Installing core dependencies...\n")

# Install quietly
!pip install -q --upgrade pip
!pip install -q numpy pandas scikit-learn scipy
!pip install -q torch torchvision transformers sentence-transformers accelerate
!pip install -q faiss-cpu lightgbm xgboost
!pip install -q Pillow opencv-python
!pip install -q langdetect nltk
!pip install -q tqdm rich pyyaml python-dotenv jsonschema
!pip install -q fastapi uvicorn pydantic
!pip install -q streamlit

print("\n✅ Dependencies installed!")
print("\n🔍 Checking versions:")

import torch
import transformers
import sentence_transformers
import faiss

print(f"  PyTorch: {torch.__version__}")
print(f"  Transformers: {transformers.__version__}")
print(f"  Sentence-Transformers: {sentence_transformers.__version__}")
print(f"  FAISS: {faiss.__version__}")
print(f"  CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")

📦 Installing core dependencies...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

✅ Dependencies installed!

🔍 Checking versions:
  PyTorch: 2.9.0+cu126
  Transformers: 4.57.3
  Sentence-Transformers: 5.1.2
  FAISS: 1.13.1
  CUDA Available: True
  GPU: NVIDIA A100-SXM4-40GB


In [8]:
# ============================================================
# 7) CREATE CORE FILES
# ============================================================

print("📝 Creating core files...\n")

# Create __init__.py files
init_files = [
    "src/__init__.py",
    "api/__init__.py",
    "api/endpoints/__init__.py",
    "ui/__init__.py",
    "ui/components/__init__.py",
    "tests/__init__.py"
]

for init_file in init_files:
    init_path = PROJECT_ROOT / init_file
    init_path.touch(exist_ok=True)
    print(f"  ✅ {init_file}")

print("\n✅ Core files created!")

📝 Creating core files...

  ✅ src/__init__.py
  ✅ api/__init__.py
  ✅ api/endpoints/__init__.py
  ✅ ui/__init__.py
  ✅ ui/components/__init__.py
  ✅ tests/__init__.py

✅ Core files created!


In [9]:
# ============================================================
# 8) INITIALIZE GIT (Optional)
# ============================================================

print("🔧 Initializing Git...\n")

# Change to project directory
%cd {PROJECT_ROOT}

# Initialize git if not already
!git init

# Set git config
!git config user.name "Your Name"
!git config user.email "your.email@example.com"

print("\n✅ Git initialized!")
print("   Don't forget to update .gitignore and add your files!")

🔧 Initializing Git...

/content/drive/MyDrive/ai_fashion_assistant_v2
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/ai_fashion_assistant_v2/.git/

✅ Git initialized!
   Don't forget to update .gitignore and add your files!


In [10]:
# ============================================================
# 9) TEST IMPORTS
# ============================================================

print("🧪 Testing imports...\n")

try:
    import numpy as np
    import pandas as pd
    import torch
    from transformers import CLIPModel, CLIPProcessor
    from sentence_transformers import SentenceTransformer
    import faiss
    import lightgbm as lgb
    from PIL import Image
    import yaml
    from pathlib import Path

    print("✅ All imports successful!")

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   Please run the installation cell again")

🧪 Testing imports...

✅ All imports successful!


In [11]:
# ============================================================
# 10) FINAL CHECKLIST
# ============================================================

print("✅ PROJECT INITIALIZATION COMPLETED!")
print("=" * 60)
print("\n📋 Checklist:")
print("  ✅ Google Drive mounted")
print("  ✅ Folder structure created")
print(f"  {'✅' if (PROJECT_ROOT / 'data/raw/styles.csv').exists() else '❌'} styles.csv present")
print(f"  {'✅' if (PROJECT_ROOT / 'data/raw/images').exists() else '❌'} Images folder present")
print("  ✅ Dependencies installed")
print("  ✅ Git initialized")
print("  ✅ Imports tested")

print("\n🚀 Next Steps:")
print("  1. Review the project structure")
print("  2. Start with Phase 1: Data Preparation")
print("  3. Open: notebooks/phase1_foundation/01_data_preparation.ipynb")

print("\n" + "=" * 60)
print("Project Root:", PROJECT_ROOT)
print("=" * 60)

✅ PROJECT INITIALIZATION COMPLETED!

📋 Checklist:
  ✅ Google Drive mounted
  ✅ Folder structure created
  ❌ styles.csv present
  ✅ Images folder present
  ✅ Dependencies installed
  ✅ Git initialized
  ✅ Imports tested

🚀 Next Steps:
  1. Review the project structure
  2. Start with Phase 1: Data Preparation
  3. Open: notebooks/phase1_foundation/01_data_preparation.ipynb

Project Root: /content/drive/MyDrive/ai_fashion_assistant_v2


---

## 📊 Project Overview

**Current Status:** ✅ Initialized

**Next Notebook:** `phase1_foundation/01_data_preparation.ipynb`

---

### 📁 Folder Structure

```
ai_fashion_assistant_v2/
├── data/              # Data files
├── embeddings/        # Generated embeddings
├── models/            # Trained models
├── indexes/           # FAISS indexes
├── src/               # Source code
├── api/               # FastAPI backend
├── ui/                # Streamlit frontend
├── configs/           # Configuration files
├── tests/             # Unit tests
└── docs/              # Documentation
```

---

### 🎯 Roadmap

- [x] **Phase 0:** Project initialization
- [ ] **Phase 1:** Foundation (Data + SSOT)
- [ ] **Phase 2:** Embeddings
- [ ] **Phase 3:** Retrieval
- [ ] **Phase 4:** Query Understanding
- [ ] **Phase 5:** Ranking
- [ ] **Phase 6:** Personalization
- [ ] **Phase 7:** Evaluation
- [ ] **Phase 8:** Chatbot
- [ ] **Phase 9:** Deployment
- [ ] **Phase 10:** Final Demo

---